# Split and Read 2.5 GB of XLSX and write to feather file format for faster subsequent read

__Motivation:__ Reading 2.5 GB of dataset as separate large Excel files into
pandas dataframe in one-go failed after 90 minutes on my Latitude XPS 15 with
32GB of RAM.
There was not enough memory to buffer and hold the data in memory.
The old approach failed before the data could be converted into feather format.

## Helper functions

In [1]:
import os
import re
import pandas as pd


def read_combine(io):
    """
    Read a list of Excel files and combine them into a panadas DataFrame
    
    Parameters
    ----------
        io (list): list of fully qualified filenames
        
    Returns
    -------
    DataFrame with filenames appended at the last column.
    """
    list_df = []
    for each in io:
        # print("...reading", os.path.basename(each)) 
        df = pd.read_excel(each)
        df["filename"] = os.path.basename(each)
        list_df.append(df)
    
    return pd.concat(list_df)


def get_abspath(path, pattern):
    """
    Search directory non-recursively for filename by name patterns
    
    Parameters
    ----------
    path : 
    pattern : 

    Returns
    -------
    a list of absolute path, use os.path.basename to get the filename
    """
    return [os.path.join(path, x) for x in os.listdir(path) if re.search(pattern, x)]


def print_filesize(abspath):
    df=pd.DataFrame({'filename': [os.path.basename(x) for x in abspath],
                     'size': [os.stat(x).st_size for x in abspath]}) \
        .set_index('filename')
    print(df)
    print("total size:", round(sum(df["size"]) / 1024**3, 2), "gb")
    return df

## Excel files

In [2]:
data_dir = os.path.join(os.getcwd(), 'data', 'wallets successful event')
wallets_fn = get_abspath(data_dir, '\.xlsx$')
fsizes = print_filesize(wallets_fn)

                                                         size
filename                                                     
alien-frens-evolution_success1.xlsx                  64074307
boredapekennelclub_success1.xlsx                    139048703
boredapeyachtclub_success1.xlsx                      61715648
clonex_success1.xlsx                                 44406751
coolcatsnft_A1.xlsx                                 241609963
coolcatsnft_A2.xlsx                                 240563942
coolcatsnft_A3.xlsx                                 101283300
coolcatsnft_AA1.xlsx                                124645577
coolcatsnft_AA11.xlsx                               125791182
coolcatsnft_AA2.xlsx                                123305452
coolcatsnft_AA22.xlsx                               126019118
coolcatsnft_補跑B.xlsx                                   868343
coolcatsnft_補跑E.xlsx                                  2109287
cryptoadz-by-gremplin_success1.xlsx                  89797994
cryptosk

# Read Excel files into pandas.DataFrame and save as feather

...read each Excel and save as feather individually before any post read processing

In [ ]:
for fn in wallets_fn:
    to_fn = os.path.join(data_dir, os.path.basename(fn)[:-4] + 'feather')

    print("... reading", os.path.basename(fn), os.stat(fn).st_size)
    df = pd.read_excel(fn)
    df["filename"] = os.path.basename(fn)
    df.reset_index(inplace=True)
    df.to_feather(to_fn)
    print("... saving", os.path.basename(to_fn), os.stat(to_fn).st_size)
    

# Post Processing

In [3]:
fs = get_abspath(data_dir, '\.feather$')
lst = (pd.read_feather(each) for each in fs)

In [4]:
df = pd.concat(lst)

In [5]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11322150 entries, 0 to 333345
Data columns (total 35 columns):
 #   Column                  Non-Null Count     Dtype  
---  ------                  --------------     -----  
 0   index                   11322150 non-null  int64  
 1   Unnamed: 0              11322150 non-null  int64  
 2   event_timestamp         11312039 non-null  object 
 3   event_type              11312039 non-null  object 
 4   token_id                11217880 non-null  object 
 5   num_sales               11217880 non-null  float64
 6   listing_time            10612413 non-null  object 
 7   token_owner_address     11217880 non-null  object 
 8   token_seller_address    11293040 non-null  object 
 9   from_account_address    3871 non-null      object 
 10  deal_price              11312025 non-null  float64
 11  payment_token_symbol    11307893 non-null  object 
 12  payment_token_decimals  11307998 non-null  float64
 13  payment_token_usdprice  11305808 non-null 

In [6]:
df = df[df.msg == "success"]
df.drop(["index", "Unnamed: 0", "pages", "msg", "FILTER", "next_param"], axis=1, inplace=True)

In [7]:
df.event_timestamp = pd.to_datetime(df.event_timestamp)
df.listing_time = pd.to_datetime(df.listing_time)
df.quantity = pd.to_numeric(df.quantity, errors='coerce')

In [8]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11312039 entries, 0 to 333345
Data columns (total 29 columns):
 #   Column                  Non-Null Count     Dtype         
---  ------                  --------------     -----         
 0   event_timestamp         11312039 non-null  datetime64[ns]
 1   event_type              11312039 non-null  object        
 2   token_id                11217880 non-null  object        
 3   num_sales               11217880 non-null  float64       
 4   listing_time            10612413 non-null  datetime64[ns]
 5   token_owner_address     11217880 non-null  object        
 6   token_seller_address    11293040 non-null  object        
 7   from_account_address    3871 non-null      object        
 8   deal_price              11312025 non-null  float64       
 9   payment_token_symbol    11307893 non-null  object        
 10  payment_token_decimals  11307998 non-null  float64       
 11  payment_token_usdprice  11305808 non-null  float64       
 12  

# Split and write to feather

In [9]:
grp = df.groupby("filename")

for name, group in grp:
    print(name)
    g=group.reset_index().drop("filename", axis=1)
    g.to_feather(os.path.join(data_dir, 'feather', os.path.basename(name)[:-4] + 'feather'), compression='lz4')

alien-frens-evolution_success1.xlsx
boredapekennelclub_success1.xlsx
boredapeyachtclub_success1.xlsx
clonex_success1.xlsx
coolcatsnft_A1.xlsx
coolcatsnft_A2.xlsx
coolcatsnft_A3.xlsx
coolcatsnft_AA1.xlsx
coolcatsnft_AA11.xlsx
coolcatsnft_AA2.xlsx
coolcatsnft_AA22.xlsx
coolcatsnft_補跑B.xlsx
coolcatsnft_補跑E.xlsx
cryptoadz-by-gremplin_success1.xlsx
cryptoskulls_success1.xlsx
cyberkongz-vx_success1.xlsx
doodlesofficial_success1.xlsx
hapeprime_success1.xlsx
kaiju-kingz_success1.xlsx
karafuru_success1.xlsx
lazy-lions_success1.xlsx
mekaverse_success1.xlsx
mfers_success1.xlsx
mutant-ape-yacht-club_success1.xlsx
phantabear_success1.xlsx
proof-moonbirds_success1.xlsx
rektguy_rarelandnft_pieceofshit_ivedoneit_success1.xlsx
world-of-women-nft_success1.xlsx


# Checking out the data

In [10]:
df.groupby(by="collection_slug", as_index=False).size().sort_values(by="size", ascending=False).head(20)

,collection_slug,size
10438,cryptokitties,87113
8749,cool-cats-nft,75452
31304,parallelalpha,72462
34418,rarible,65798
6191,boredapeyachtclub,50417
27935,mutant-ape-yacht-club,50392
23502,lazy-lions,45897
33610,pudgypenguins,44149
45791,world-of-women-nft,41750
9891,cryptoadz-by-gremplin,38680


In [11]:
top20_nft = df.groupby(by="collection_slug").size().sort_values(ascending=False).iloc[:20].index
top20_nft

Index(['cryptokitties', 'cool-cats-nft', 'parallelalpha', 'rarible',
       'boredapeyachtclub', 'mutant-ape-yacht-club', 'lazy-lions',
       'pudgypenguins', 'world-of-women-nft', 'cryptoadz-by-gremplin',
       'deadfellaz', 'creatureworld', 'robotos-official', 'adam-bomb-squad',
       'bored-ape-kennel-club', 'phantabear', 'thewickedcraniums',
       'ape-gang-old', 'axie', 'cyberkongz-vx'],
      dtype='object', name='collection_slug')